In [1]:
from c2048 import Game, random_play, push
from DDQN_keras_CNNs import DoubleDQNAgent
import matplotlib.pyplot as plt
import numpy as np
import math
from copy import deepcopy

Using TensorFlow backend.


In [2]:
# convert the input game matrix into corresponding power of 2 matrix.
def transform_input(X):
    power_mat = np.zeros(shape=(1, 4, 4, 16), dtype=np.float32)
    for i in range(4):
        for j in range(4):
            if X[i][j] == 0:
                power_mat[0][i][j][0] = 1.0
            else:
                power = int(math.log(X[i][j], 2))
                power_mat[0][i][j][power] = 1.0
    return power_mat


# find the number of empty cells in the game matrix.
def findEmptyCell(mat):
    count = 0
    for i in range(len(mat)):
        for j in range(len(mat)):
            if mat[i][j] == 0:
                count += 1
    return count

In [ ]:
# 2048 game
env = Game()
'''
DQNAgent = MyDQN(env.n_actions, env.n_features,
                  learning_rate=0.01,
                  reward_decay=0.9,
                  e_greedy=0.99,
                  replace_target_iter=1000,
                  memory_size=50000,
                  e_greedy_increment=0.001,
                  double_q=True,
                  dueling=True,
                  # output_graph=True
                  )
'''

# Training
final_score = []
steps = 0
EPISODES = 200001
update_target_step = 10

# agent = DoubleDQNAgent(env.n_features[0], env.n_actions, 6000)
agent = DoubleDQNAgent(env.n_features[0], env.n_actions)

try:
    agent.load_weights('myDQN_2048.h5')
    print("Loaded weight file")
except:
    pass

scores, episodes = [], []

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 4, 4, 16)      0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 4, 3, 128)     4224        input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_4 (Conv2D)                (None, 3, 4, 128)     4224        input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 4, 2, 128)     32896       conv2d_1[0][0]                   
___________________________________________________________________________________________

In [ ]:
for episode in range(EPISODES):
    score = 0
    state = env.reset()

    # state = np.log(state + 1)
    # state = np.reshape([state], (4, 4, 1))
    # state = np.array([state])

    # State Transformation
    state_tmp = deepcopy(state)
    state = transform_input(state)
    state = np.array(state, dtype=np.float32).reshape(1, 4, 4, 16)

    done = False
    while not done:
        # if agent.render:
        # if True:
        #    env.render()

        # get action for the current state and go one step in environment
        action_list = agent.get_action(state)
        i = 0
        for action in action_list:
            next_state, reward, done, info = env.step(action)
            if done:
                break
            if np.array_equal(state_tmp, next_state):
                if i < 3:
                    i += 1
                else:
                    print("ERROR!")
                continue
            else:
                break

        # Reward Adjustment
        # 1. Empty Cell Reward
        if not done:
            empty_prev = findEmptyCell(state_tmp)
            empty_next = findEmptyCell(next_state)
            reward_empty = empty_next + 1 - empty_prev  # See the difference of empty cell
        else:
            reward_empty = 0

            # 2. Next Max Cell Reward
        prev_max = np.max(state_tmp)
        next_max = np.max(next_state)
        if prev_max != next_max:
            reward_nextMax = math.log(next_max, 2) * 0.1
        else:
            reward_nextMax = 0
        reward_final = reward_empty + reward_nextMax

        # Save this for next reward checking
        state_tmp = deepcopy(next_state)

        # State Transformation
        next_state = transform_input(next_state)
        next_state = np.array(next_state, dtype=np.float32).reshape(1, 4, 4, 16)

        # save the sample <s, a, r, s'> to the replay memory
        agent.append_sample(state, action, reward_final, next_state, done)

        state = next_state
        steps += 1

        # Update epsilon
        if episode > 10000 or (agent.epsilon > 0.1 and steps % 2500 == 0):
            agent.epsilon = agent.epsilon * agent.epsilon_decay

        if done:
            # every episode do the training
            agent.train_model()

            if episode % 20 == 0:
                # every 10 episode update the target model to be same with model
                agent.update_target_model()

            # every episode, plot the play time
            score = env.score
            scores.append(score)
            episodes.append(episode)
            print("episode:", episode, "  score:", score, "  memory length:",
                  len(agent.memory), "  epsilon:", agent.epsilon)
            if episode % 100 == 0:
                env.display()

print("Done")


agent.save_weights('myDQN_2048.h5', overwrite=True)

episode: 0   score: 680.0   memory length: 88   epsilon: 1.0

+------+------+------+------+
| 2.0  | 4.0  | 32.0 | 2.0  |
+------+------+------+------+
| 16.0 | 64.0 | 2.0  | 4.0  |
+------+------+------+------+
| 4.0  | 32.0 | 16.0 | 8.0  |
+------+------+------+------+
| 8.0  | 2.0  | 8.0  | 4.0  |
+------+------+------+------+
episode: 1   score: 1396.0   memory length: 234   epsilon: 1.0
episode: 2   score: 388.0   memory length: 301   epsilon: 1.0
episode: 3   score: 564.0   memory length: 380   epsilon: 1.0
episode: 4   score: 1272.0   memory length: 519   epsilon: 1.0
episode: 5   score: 848.0   memory length: 620   epsilon: 1.0
episode: 6   score: 1304.0   memory length: 756   epsilon: 1.0
episode: 7   score: 1104.0   memory length: 875   epsilon: 1.0
episode: 8   score: 304.0   memory length: 933   epsilon: 1.0
episode: 9   score: 832.0   memory length: 1042   epsilon: 1.0
episode: 10   score: 632.0   memory length: 1125   epsilon: 1.0
episode: 11   score: 972.0   memory lengt